# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import pandas as pd
import requests
from pprint import pprint
import gmaps
import random

# Google developer API key
from api_keys import gkey
from api_keys import geoapify_key

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities_personal.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,broome,42.2506,-75.8330,293.96,81,100,5.81,US,1667743739
1,1,lata,40.1629,-8.3327,292.08,84,100,0.53,PT,1667743752
2,2,cockburn town,21.4612,-71.1419,299.28,75,100,6.29,TC,1667743753
3,3,hithadhoo,-0.6000,73.0833,300.46,75,100,3.00,MV,1667743753
4,4,khor,47.8861,134.9464,269.41,73,9,2.57,RU,1667743754


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
# get coordinates into a list of tuples
coordinates = []
sizes = []
for index,value in city_data_df.iterrows():
    coordinates.append((value['Lat'], value['Lng']))
    sizes.append(value['Humidity'])


colors = [random.choice(['red', 'green', 'blue', 'purple', 'yellow', 'teal']) for coordinate in coordinates]

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# convert temp
city_data_df_tempadj = city_data_df
city_data_df_tempadj['Max Temp'] = city_data_df['Max Temp'] * 1.8 - 459.67


# Narrow down cities that fit criteria and drop any results with null values
best_cities_df = city_data_df_tempadj.loc[(city_data_df_tempadj['Max Temp'] >= 70)\
    & (city_data_df_tempadj['Max Temp'] <= 80)\
        & (city_data_df_tempadj['Humidity'] >= 50)\
            & (city_data_df_tempadj['Humidity'] <= 60)\
                & (city_data_df_tempadj['Cloudiness'] == 0)]



In [23]:
# Drop any rows with null values
best_cities_df.dropna()

# display best cities
best_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
265,265,san javier,37.8063,-0.8374,78.980,54,0,3.60,ES,1667743880
287,287,namibe,-15.1961,12.1522,77.378,52,0,5.64,AO,1667743892
466,466,bundu,23.1833,85.5833,75.416,60,0,0.92,IN,1667743976
521,521,nouadhibou,20.9310,-17.0347,74.642,53,0,4.25,MR,1667744002


In [24]:
# Display sample data
# get coordinates into a list of tuples
coordinates = []
for index,value in best_cities_df.iterrows():
    coordinates.append((value['Lat'], value['Lng']))

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = best_cities_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = pd.Series([], dtype='string')

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
265,265,san javier,37.8063,-0.8374,78.980,54,0,3.60,ES,1667743880,<NA>
287,287,namibe,-15.1961,12.1522,77.378,52,0,5.64,AO,1667743892,<NA>
466,466,bundu,23.1833,85.5833,75.416,60,0,0.92,IN,1667743976,<NA>
521,521,nouadhibou,20.9310,-17.0347,74.642,53,0,4.25,MR,1667744002,<NA>


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 5000 # in meters
params = {"apiKey":geoapify_key}    
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = "hotel"
    params["bias"] = f"circle:{lng},{lat},{radius}"
    # filter = f"circle:-87.770231,41.878968,{radius}"
    # Set base URL
    # base_url = f"https://api.geoapify.com/v2/places?{params}"
    base_url = f"https://api.geoapify.com/v2/places?apiKey={geoapify_key}&filter=circle:{lng},{lat},{radius}&categories=accommodation.hotel"

    response = requests.get(base_url)
    # pprint(response.json())
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index,"Hotel Name"] = response.json()["features"][0]["properties"]['datasource']['raw']["name"]
        # print(response.json()["features"][0]["properties"]['datasource']['raw']["name"])
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Apartahotel Bahía
san javier - nearest hotel: Apartahotel Bahía
IKA Hotel Namibe
namibe - nearest hotel: IKA Hotel Namibe
bundu - nearest hotel: No hotel found
السلام
nouadhibou - nearest hotel: السلام


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
265,265,san javier,37.8063,-0.8374,78.980,54,0,3.60,ES,1667743880,Apartahotel Bahía
287,287,namibe,-15.1961,12.1522,77.378,52,0,5.64,AO,1667743892,IKA Hotel Namibe
466,466,bundu,23.1833,85.5833,75.416,60,0,0.92,IN,1667743976,No hotel found
521,521,nouadhibou,20.9310,-17.0347,74.642,53,0,4.25,MR,1667744002,السلام


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
names = []
for index,value in hotel_df.iterrows():
    names.append(f"{value['Hotel Name']}, {value['Country']}")

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
symbol_layer = gmaps.symbol_layer(coordinates, hover_text=names)

# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(symbol_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…